In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
df_payments_notref_android = pd.read_csv(r'data/180321_notref_android.csv')
df_payments_ref_android = pd.read_csv(r'data/180321_ref_android.csv')
df_payments_notref_ios= pd.read_csv(r'data/180321_notref_ios.csv')
df_payments_ref_ios = pd.read_csv(r'data/180321_ref_ios.csv')

In [3]:
df_payments_notref_android['ref'] = 'notref'
df_payments_ref_android['ref'] = 'ref'
df_payments_notref_ios['ref'] = 'notref'
df_payments_ref_ios['ref'] = 'ref'

df_payments_notref_android['app'] = 'android'
df_payments_ref_android['app'] = 'android'
df_payments_notref_ios['app'] = 'ios'
df_payments_ref_ios['app'] = 'ios'

df_payments = df_payments_notref_android.append(df_payments_ref_android).append(df_payments_notref_ios).append(df_payments_ref_ios)

In [4]:
# https://portraits.core.tf/goto/1023ade78fbeb792c6e9e45b91c2a294
df_installs_all = pd.read_csv(r'data/180321_installs.csv',thousands=',')
df_installs_all.columns = ['created per day','ref','app','users']
df_installs_all['created per day'] = df_installs_all['created per day'].map(lambda x: x[:10])
dict_app = {'first_app:topfacemobappandroid': 'android','first_app:topfacemobappios':'ios'}
dict_ref = {'referral:0': 'notref','referral:1':'ref'}
df_installs_all['app'] = df_installs_all['app'].map(dict_app)
df_installs_all['ref'] = df_installs_all['ref'].map(dict_ref)

In [5]:
df_payments['days from payment'] = df_payments['payment date'].map(lambda x: datetime.strptime(x[:10], "%Y-%m-%d")) - df_payments['registration date'].map(lambda x: datetime.strptime(x[:10], "%Y-%m-%d"))
df_payments['days from payment'] = df_payments['days from payment'].map(lambda x: x.days)
df_payments['registration day'] = df_payments['registration date'].map(lambda x: x[:10])
df_payments = df_payments[df_payments['days from payment'] <= 7]

In [6]:
writer = pd.ExcelWriter('out/revenue_by_days.xlsx')
for app in ['android','ios']:
    for ref in ['notref','ref']:
        df = df_payments[(df_payments['app'] == app) & (df_payments['ref'] == ref)]
        df_installs = df_installs_all[(df_installs_all['app'] == app) & (df_installs_all['ref'] == ref)]
        df_installs_2 = df_installs[['users']]
        df_installs_2.index = df_installs['created per day']
        m = df.pivot_table(index='registration day', columns='days from payment', values='revenue in cents', aggfunc=sum).fillna(0).applymap(float)
        daily_metrics = pd.concat([m,df_installs_2], join='outer', axis = 1)
        # daily_metrics.to_csv('out_' + ref + '_' + app + '.csv')
        daily_metrics.to_excel(writer,ref+'_'+app)
writer.save()